In [1]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

import plotly.graph_objs as go
from chart_studio.plotly import plot, iplot
import cufflinks as cf
cf.go_offline(connected=True)

In [8]:
data_raw = pd.read_excel('./data/01_제공데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터.xlsx')
rating_df = pd.read_excel('./data/01_제공데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_시청률 데이터.xlsx')
test_df = pd.read_excel('./data/02_평가데이터/2020 빅콘테스트 데이터분석분야-챔피언리그_2020년 6월 판매실적예측데이터(평가데이터).xlsx')

In [11]:
def data_preprocessing(df):
    df.columns = df.iloc[0,:]
    df = df.drop(df.index[0])
    
    df['노출(분)'].fillna(0,inplace = True) #노출(분) 결측치 0으로
    df = df[df['상품군'] != '무형'] #무형상품 일단제외
    
    df['판매수량'] = df['취급액'] // df['판매단가']
    df.iloc[:,-3:] = df.iloc[:,-3:].apply(pd.to_numeric)
    
    df = df.reset_index(drop = True)
    
    for i in range(1,len(df['노출(분)']+1)):
        if df['노출(분)'][i] == 0:
            df['노출(분)'][i] += df['노출(분)'][i-1]

    df['노출(분)'] = list(map(lambda x: int(round(x)), df['노출(분)'])) #정수처리
    
    return df

# df = data_preprocessing(data_raw)

In [12]:
# test_df.columns = test_df.iloc[0,:]
# test_df = test_df.drop(test_df.index[0])
test_df = data_preprocessing(test_df)

In [14]:
def comp_products(df,col):
    products = [df[df.상품군 == i] for i in list(set(df.상품군))]
    sales_ls = []
    for idx,i in enumerate(list(set(df.상품군))):
        sales_ls.append(round(products[idx][col].mean()))

    return pd.DataFrame({f"평균{col}":sales_ls},index = list(set(df.상품군))).iplot(kind = 'bar')
    
    

In [15]:
def describe_sorts(df_,col):
    df_ = df[df['상품군'] == col]
    print(f"전체 {df_.shape[0]}개의 상품 중 유니크한 상품 수는 {df_.상품명.nunique()}개이다.")
    print(f"마더코드의 개수는 {df_.마더코드.nunique()}개이다.")
    print(f"판매단가의 평균은 {round(df_.판매단가.mean())}원, 최소 {df_.판매단가.min()}원, 최대 {df_.판매단가.max()}원이다.")
    print(f"판매수량의 평균은 {round(df_.판매수량.mean())}개, 최소 {(df_.판매수량.min())}개, 최대 {round(df_.판매수량.max())}개이다.")
    
    return df_.판매수량.iplot()

In [16]:
comp_products(df,'판매단가')

In [17]:
comp_products(df,'판매수량')

#### 가구, 가전 , 건강기능 에 대한 카테고리화

In [18]:
furniture = df[df.상품군 == '가구'].reset_index()
appliance = df[df.상품군 == '가전'].reset_index()
healthcare = df[df.상품군 == '건강기능'].reset_index()
print(furniture.shape, appliance.shape, healthcare.shape)

(2614, 10) (6479, 10) (790, 10)


## 가구

In [19]:
furniture.판매수량.describe() #평균 16개 

count    2614.000000
mean       16.926549
std        20.382807
min         0.000000
25%         2.000000
50%        10.000000
75%        24.000000
max       232.000000
Name: 판매수량, dtype: float64

In [20]:
describe_sorts(df,'가구')

전체 2614개의 상품 중 유니크한 상품 수는 102개이다.
마더코드의 개수는 14개이다.
판매단가의 평균은 843476.0원, 최소 89000원, 최대 3990000원이다.
판매수량의 평균은 17.0개, 최소 0개, 최대 232개이다.


In [21]:
furniture[furniture.판매수량 > 100] #소파 , 침대 , 서랍장

,index,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,판매수량
33,3684,2019-02-08 21:40:00,20,100205,200733,보루네오 루나 유로탑 멀티수납형 LED 침대 SS 슈퍼싱글,가구,548000,63277000,115
159,5108,2019-02-22 21:40:00,20,100205,200733,보루네오 루나 유로탑 멀티수납형 LED 침대 SS 슈퍼싱글,가구,548000,58879000,107
1926,26609,2019-09-13 01:00:00,20,100230,200793,뉴 벨라홈 리클라이닝 소파,가구,89000,9409000,105
1927,26610,2019-09-13 01:20:00,20,100230,200793,뉴 벨라홈 리클라이닝 소파,가구,89000,20675000,232
1928,26611,2019-09-13 01:40:00,17,100230,200793,뉴 벨라홈 리클라이닝 소파,가구,89000,17527000,196
1990,27334,2019-09-19 12:40:00,20,100206,200740,보루네오 심플 광폭 서랍장 1000 5단,가구,199000,29832000,149
2050,28223,2019-09-28 01:20:00,20,100230,200793,뉴 벨라홈 리클라이닝 소파,가구,89000,17520000,196
2051,28224,2019-09-28 01:40:00,17,100230,200793,뉴 벨라홈 리클라이닝 소파,가구,89000,19073000,214
2076,28611,2019-10-01 20:40:00,20,100205,200735,보루네오 루나시즌2 유로탑 멀티수납형 LED침대 슈퍼싱글,가구,548000,57632000,105
2086,28742,2019-10-03 00:40:00,20,100206,200740,보루네오 심플 광폭 서랍장 1000 5단,가구,199000,30828000,154


### 상품명 대분류
* 침대, 소파 , 붙박이장 , 서랍장 , 시공패키지


* 침대 (SS,Q,K) : 침대가 차지하는 비중이 많아서 사이즈별로 소분류가 가능해보임


* 시공패키지는 수량은 그렇게 많지 않으나 가격이 높다.


* '리클라이너'는 소파와 함께 나오는 상품들도 있어서 소파로 분류하는게 맞아 보임. 가격대도 비슷


* 기타 : 베드룸세트 + 거실장, 흙카우치, 한샘 하이바스 


* 한샘, 보루네오 등 브랜드별 grouping이 필요한가?

## 가전

In [22]:
describe_sorts(df,'가전')

전체 6479개의 상품 중 유니크한 상품 수는 165개이다.
마더코드의 개수는 22개이다.
판매단가의 평균은 1631382.0원, 최소 164000원, 최대 3190000원이다.
판매수량의 평균은 15.0개, 최소 0개, 최대 339개이다.


In [23]:
appliance[appliance.판매수량 > 100] #세탁기 , 청소기

,index,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,판매수량
2,46,2019-01-01 20:20:00,20,100150,200533,일시불 LG 통돌이 세탁기,가전,439000,56859000,129
3,47,2019-01-01 20:20:00,20,100150,200465,무이자 LG 통돌이 세탁기,가전,489000,58685000,120
4,48,2019-01-01 20:40:00,20,100150,200533,일시불 LG 통돌이 세탁기,가전,439000,68320000,155
52,326,2019-01-04 19:40:00,20,100150,200533,일시불 LG 통돌이 세탁기,가전,439000,45431000,103
80,516,2019-01-06 21:20:00,20,100150,200533,일시불 LG 통돌이 세탁기,가전,439000,48264000,109
...,...,...,...,...,...,...,...,...,...,...
6466,36873,2019-12-25 22:00:00,20,100150,200427,LG전자 통돌이 세탁기 TR14WK1(화이트),가전,489000,144212000,294
6468,37152,2019-12-29 11:40:00,20,100150,200427,LG전자 통돌이 세탁기 TR14WK1(화이트),가전,489000,87404000,178
6469,37153,2019-12-29 12:00:00,20,100150,200427,LG전자 통돌이 세탁기 TR14WK1(화이트),가전,489000,125019000,255
6471,37259,2019-12-30 21:40:00,20,100150,200427,LG전자 통돌이 세탁기 TR14WK1(화이트),가전,489000,85156000,174


In [24]:
pd.DataFrame(appliance.groupby(appliance['상품명'])['노출(분)'].sum()).head(50)

,노출(분)
상품명,
(1등급)221L_딤채김치냉장고,520
(1등급)221L_딤채김치냉장고(LDL22DHWJJ),260
(1등급)467L_딤채김치냉장고,360
(삼성카드 6월 5%)무이자 LG 울트라HD TV 75UK6200KNB,120
221L_딤채김치냉장고,60
467L_딤채김치냉장고,180
LG 울트라HD TV 55UM7900BNA,600
LG 울트라HD TV 65UM7900BNA,600
LG 울트라HD TV 75UM7900BNA,600


In [ ]:
test_df[test_df['상품군'] == '']

In [25]:
def describe_sorts(df_,col):
    df_ = df[df['상품군'] == col]
    print(f"전체 {df_.shape[0]}개의 상품 중 유니크한 상품 수는 {df_.상품명.nunique()}개이다.")
    print(f"마더코드의 개수는 {df_.마더코드.nunique()}개이다.")
    print(f"판매단가의 평균은 {round(df_.판매단가.mean())}원, 최소 {df_.판매단가.min()}원, 최대 {df_.판매단가.max()}원이다.")
    print(f"판매수량의 평균은 {round(df_.판매수량.mean())}개, 최소 {(df_.판매수량.min())}개, 최대 {round(df_.판매수량.max())}개이다.")
    
    return df_.판매수량.iplot()

## 건강기능

In [36]:
describe_sorts(df,"건강기능")

전체 790개의 상품 중 유니크한 상품 수는 48개이다.
마더코드의 개수는 36개이다.
판매단가의 평균은 83324.0원, 최소 39900원, 최대 238000원이다.
판매수량의 평균은 372.0개, 최소 0개, 최대 3607개이다.


In [48]:
addddd = df[df['상품군'] == '농수축']

In [60]:
describe_sorts(df,'건강기능')

전체 790개의 상품 중 유니크한 상품 수는 48개이다.
마더코드의 개수는 36개이다.
판매단가의 평균은 83324.0원, 최소 39900원, 최대 238000원이다.
판매수량의 평균은 372.0개, 최소 0개, 최대 3607개이다.


In [59]:
describe_sorts(df,'농수축')

전체 3884개의 상품 중 유니크한 상품 수는 140개이다.
마더코드의 개수는 102개이다.
판매단가의 평균은 46420.0원, 최소 28900원, 최대 109900원이다.
판매수량의 평균은 972.0개, 최소 34개, 최대 6457개이다.


In [70]:
tf = test_df[test_df['상품군'] == '농수축']
pd.DataFrame(tf.groupby(tf['상품명'])['노출(분)'].sum())

,노출(분)
상품명,
(특대사이즈) 안동간고등어 13팩,60
*[맥널티] 싱글 오리진 핸드드립 커피 5종 10박스 + 커피포트 1개,180
*[맥널티] 싱글 오리진 핸드드립 커피 5종 17박스 + 커피포트 2개,180
2020 안동간고등어 20팩,120
2020 제주 햇 초당 옥수수 20개,60
...,...
한입직화장어구이10팩,120
해뜰찬별미김치7kg,60
해뜰찬포기김치10kg,60


In [68]:
af = pd.DataFrame(addddd.groupby(addddd['상품명'])['노출(분)'].sum())
af.sort_values('노출(분)',ascending=False).tail(50)

,노출(분)
상품명,
바다먹자 국내산 손질문어 9팩,120
뽕셰프 이봉원 특갈비탕 12팩,120
거창특등급사과 5kg+5kg,120
영산포숙성 홍어 9팩,120
AAC한우소머리곰탕양곰탕세트,120
AAC 한우소머리양곰탕,120
예천청결 고춧가루 2kg,120
AAC 한우소머리 곰탕,120
고창 꿀 고구마 10kg,105


In [67]:
health_sum = pd.DataFrame(healthcare.groupby(healthcare['상품명'])['노출(분)'].sum())
health_sum

,노출(분)
상품명,
뉴트리원 다이어트 구미,1200
블랙모어스 알티지 오메가3 3병 (53주분),180
서울대 더 리얼 신바이오틱스 6박스 (6개월분),60
한국인 모유유래 다이어트 유산균 BNR17 비에날씬 12주(3박스),237
""" 100%리얼 착즙 석류 "", 총 4박스",240
(무)100%리얼착즙석류 4박스,240
(무)모나코사놀,60
(일)100%리얼착즙석류 4박스,240
(일)모나코사놀,60


In [43]:
healthcare[healthcare['상품명'] == '무이자 종근당건강 락토핏 생유산균골드 단하루'].head(50)

,index,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,판매수량
145,8002,2019-03-23 07:00:00,20,100654,201977,무이자 종근당건강 락토핏 생유산균골드 단하루,건강기능,79000,6300000,79
147,8004,2019-03-23 07:20:00,20,100654,201977,무이자 종근당건강 락토핏 생유산균골드 단하루,건강기능,79000,12071000,152
149,8006,2019-03-23 07:40:00,20,100654,201977,무이자 종근당건강 락토핏 생유산균골드 단하루,건강기능,79000,27290000,345
170,11302,2019-04-21 07:00:00,20,100654,201977,무이자 종근당건강 락토핏 생유산균골드 단하루,건강기능,79000,7882000,99
172,11304,2019-04-21 07:20:00,20,100654,201977,무이자 종근당건강 락토핏 생유산균골드 단하루,건강기능,79000,13900000,175
174,11306,2019-04-21 07:40:00,20,100654,201977,무이자 종근당건강 락토핏 생유산균골드 단하루,건강기능,79000,14849000,187
176,11323,2019-04-21 11:00:00,20,100654,201977,무이자 종근당건강 락토핏 생유산균골드 단하루,건강기능,79000,9147000,115
178,11325,2019-04-21 11:20:00,20,100654,201977,무이자 종근당건강 락토핏 생유산균골드 단하루,건강기능,79000,11216000,141
180,11327,2019-04-21 11:40:00,20,100654,201977,무이자 종근당건강 락토핏 생유산균골드 단하루,건강기능,79000,12215000,154
182,11359,2019-04-21 17:00:00,20,100654,201977,무이자 종근당건강 락토핏 생유산균골드 단하루,건강기능,79000,4944000,62


In [38]:
test_df[test_df['상품군']== '건강기능']

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,판매수량
234,2020-06-04 00:20:00,20,100057,200143,뉴트리원 다이어트 구미 시즌2,건강기능,59000,NaN,NaN
235,2020-06-04 00:40:00,20,100057,200143,뉴트리원 다이어트 구미 시즌2,건강기능,59000,NaN,NaN
236,2020-06-04 01:00:00,20,100057,200143,뉴트리원 다이어트 구미 시즌2,건강기능,59000,NaN,NaN
1773,2020-06-20 00:20:00,20,100057,200143,뉴트리원 다이어트 구미 시즌2,건강기능,59000,NaN,NaN
1774,2020-06-20 00:40:00,20,100057,200143,뉴트리원 다이어트 구미 시즌2,건강기능,59000,NaN,NaN
1775,2020-06-20 01:00:00,20,100057,200143,뉴트리원 다이어트 구미 시즌2,건강기능,59000,NaN,NaN
2112,2020-06-23 17:20:00,20,100654,201979,일시불 종근당건강 락토핏 생유산균골드 단하루,건강기능,75050,NaN,NaN
2113,2020-06-23 17:20:00,20,100654,201976,무이자 종근당건강 락토핏 생유산균골드 단하루,건강기능,79000,NaN,NaN
2114,2020-06-23 17:40:00,20,100654,201979,일시불 종근당건강 락토핏 생유산균골드 단하루,건강기능,75050,NaN,NaN
2115,2020-06-23 17:40:00,20,100654,201976,무이자 종근당건강 락토핏 생유산균골드 단하루,건강기능,79000,NaN,NaN


In [28]:
data_raw[data_raw['상품군'] == '건강기능'].head(50)

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
71,2019-01-02 07:00:00,20,100315,201002,(직매입) 종근당건강 아이클리어 루테인,건강기능,69000,14944000
72,2019-01-02 07:20:00,20,100315,201002,(직매입) 종근당건강 아이클리어 루테인,건강기능,69000,25881000
73,2019-01-02 07:40:00,20,100315,201002,(직매입) 종근당건강 아이클리어 루테인,건강기능,69000,30126000
192,2019-01-03 12:00:00,20,100315,201002,(직매입) 종근당건강 아이클리어 루테인,건강기능,69000,18481000
193,2019-01-03 12:20:00,20,100315,201002,(직매입) 종근당건강 아이클리어 루테인,건강기능,69000,24779000
194,2019-01-03 12:40:00,20,100315,201002,(직매입) 종근당건강 아이클리어 루테인,건강기능,69000,45269000
267,2019-01-04 06:00:00,20,100651,201972,제주농장 유기 양배추진액 120포,건강기능,74900,10439000
268,2019-01-04 06:20:00,20,100651,201972,제주농장 유기 양배추진액 120포,건강기능,74900,17055000
269,2019-01-04 06:40:00,20,100651,201972,제주농장 유기 양배추진액 120포,건강기능,74900,43680000
448,2019-01-06 06:00:00,20,100655,201981,종근당건강 전립쎈 프라임,건강기능,89000,7549000


### Test data

**train data의 노출(분)로 정렬했을 때 상위 4개의 품목 중 3개가 train 임**

테스트 데이터의 경우 건강기능 상품군의 종류가 2개밖에없음.

In [29]:
test_df[test_df.상품군 == '건강기능']

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,판매수량
234,2020-06-04 00:20:00,20,100057,200143,뉴트리원 다이어트 구미 시즌2,건강기능,59000,NaN,NaN
235,2020-06-04 00:40:00,20,100057,200143,뉴트리원 다이어트 구미 시즌2,건강기능,59000,NaN,NaN
236,2020-06-04 01:00:00,20,100057,200143,뉴트리원 다이어트 구미 시즌2,건강기능,59000,NaN,NaN
1773,2020-06-20 00:20:00,20,100057,200143,뉴트리원 다이어트 구미 시즌2,건강기능,59000,NaN,NaN
1774,2020-06-20 00:40:00,20,100057,200143,뉴트리원 다이어트 구미 시즌2,건강기능,59000,NaN,NaN
1775,2020-06-20 01:00:00,20,100057,200143,뉴트리원 다이어트 구미 시즌2,건강기능,59000,NaN,NaN
2112,2020-06-23 17:20:00,20,100654,201979,일시불 종근당건강 락토핏 생유산균골드 단하루,건강기능,75050,NaN,NaN
2113,2020-06-23 17:20:00,20,100654,201976,무이자 종근당건강 락토핏 생유산균골드 단하루,건강기능,79000,NaN,NaN
2114,2020-06-23 17:40:00,20,100654,201979,일시불 종근당건강 락토핏 생유산균골드 단하루,건강기능,75050,NaN,NaN
2115,2020-06-23 17:40:00,20,100654,201976,무이자 종근당건강 락토핏 생유산균골드 단하루,건강기능,79000,NaN,NaN


In [30]:
df[df.상품명 == '뉴트리원 다이어트 구미 시즌2']

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,판매수량
33304,2019-11-19 08:20:00,20,100057,200144,뉴트리원 다이어트 구미 시즌2,건강기능,79000,11193000,141
33305,2019-11-19 08:40:00,20,100057,200144,뉴트리원 다이어트 구미 시즌2,건강기능,79000,17766000,224
33306,2019-11-19 09:00:00,20,100057,200144,뉴트리원 다이어트 구미 시즌2,건강기능,79000,22422000,283
37205,2019-12-30 00:20:00,20,100057,200142,뉴트리원 다이어트 구미 시즌2,건강기능,69000,14312000,207
37206,2019-12-30 00:40:00,20,100057,200142,뉴트리원 다이어트 구미 시즌2,건강기능,69000,22013000,319
37207,2019-12-30 01:00:00,20,100057,200142,뉴트리원 다이어트 구미 시즌2,건강기능,69000,28592000,414


**똑같은 제품이어도 판매단가가 떨어지면 많이산다?** 

11월보다 12월에 만원 할인된 금액으로 판매하였고 판매수량이 지속적으로 증가하는 것을 알 수 있음.

좋다고 소문나서?,  더 낮시간대에 방송했으니깐? , 기존보다 가격이 싸졌으니깐? (과연 이걸 인지하는가?)

만약 후자가 맞다면 기존보다 가격대가 떨어진 제품이 test data에 포함되어있다면 예상판매수량에 가중치를 줘야맞지않나


어찌됐든 눈으로 보고 분류했을 때 대분류는

**루테인 , 양배추 , 두유 , 홍삼, 석류 ,  오메가3 , 종근당건강,  레모나,
바이오틱스 , 뉴트리원 다이어트 구미**

# Keywords search

In [31]:
import re

In [32]:
def indexing_category(df,col,*kwargs):
    df = df[df.상품군 == col]
    keywords = [x for x in kwargs]
    idx_ls, df_ls = [],[]
    patterns = [re.compile(x) for x in keywords]
    for x in patterns:
        for idx,i in enumerate(df.상품명):
            match = re.search(x,i)
            if match:
                idx_ls.append(idx)
            else:
                pass
        df_ls.append(df.iloc[idx_ls,:])
    
    return df_ls[-1]

In [107]:
def indexing_category(df,col,*kwargs):
    df = df[df.상품군 == col]
    keywords = [x for x in kwargs]
    idx_ls, df_ls = [],[]
    patterns = [re.compile(x) for x in keywords]
    for x in patterns:
        for idx,i in enumerate(df.상품명):
            match = re.search(x,i)
            if match:
                idx_ls.append(idx)
            else:
                pass
        df_ = df.iloc[idx_ls,:]
        df_.reset_index(drop= True)
        df_ls.append(df_)
    
    return df_ls

* 건강기능 : '루테인','양배추','두유','홍삼','석류','유산균','뉴트리원'


* 가전 : '세탁기', '냉장고', 'TV', '노트북', '공기청정기', '건조기', '청소기', '에어컨', '스타일러'


* 가구 : '침대', '소파', '붙박이장','서랍','시공패키지'

In [113]:
def indexing(df,col,x):
    df = df[df.상품군 == col]
    idx_ls = []
    for idx,i in enumerate(df.상품명):
        match = re.search(re.compile(x),i)
        if match:
            idx_ls.append(idx)
        else:
            pass
    
    df_ = df.iloc[idx_ls,:]
    df_['세부품목'] = x
    
    return df_

In [120]:
건강기능 = pd.concat([indexing(df,'건강기능','루테인'),indexing(df,'건강기능','양배추'),indexing(df,'건강기능','두유'),
          indexing(df,'건강기능','홍삼'),indexing(df,'건강기능','석류'),indexing(df,'건강기능','유산균'),indexing(df,'건강기능','뉴트리원')])

In [125]:
가구 = pd.concat([indexing(df,'가구','침대'),indexing(df,'가구','소파'),indexing(df,'가구','붙박이장'),indexing(df,'가구','서랍')])

In [127]:
가전 = pd.concat([indexing(df,'가전','세탁기'),indexing(df,'가전','냉장고'),indexing(df,'가전','TV'),indexing(df,'가전','노트북'),
                indexing(df,'가전','공기청정기'),indexing(df,'가전','건조기'),indexing(df,'가전','청소기'),indexing(df,'가전','에어컨'),
                indexing(df,'가전','스타일러')]

In [134]:
가전_가구_건강 = pd.concat([가전,가구,건강기능]).reset_index(drop = True)

In [136]:
가전_가구_건강.to_csv('./data/가전_가구_건강.csv')